In [114]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib
import matplotlib.pyplot as plt
# Import the API key.
from config import g_key

In [115]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")


In [59]:
# Ask the customer if they want a location with rain or snow.
rain = str(input("Do you want it to be raining? (yes/no) "))
snow = str(input("Do you want it to be snowing? (yes/no) "))

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [60]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 80
What is the maximum temperature you would like for your trip? 90


In [69]:
# Filters the dataset to find the cities that fit in the criteria.
customer_cities_df = []
if (rain == 'yes') & (snow == 'yes'):
    customer_cities_df = city_data_df[(city_data_df['Rain Inches (last 3 hrs)'] > 0) | (city_data_df['Snow Inches (last 3 hrs)'] > 0)]
elif (rain == 'no') & (snow == 'yes'):
    customer_cities_df = city_data_df[(city_data_df['Rain Inches (last 3 hrs)'] == 0) | (city_data_df['Snow Inches (last 3 hrs)'] > 0)]
elif (rain == 'yes') & (snow == 'no'):
    customer_cities_df = city_data_df[(city_data_df['Rain Inches (last 3 hrs)'] > 0) | (city_data_df['Snow Inches (last 3 hrs)'] == 0)]
elif (rain == 'no') & (snow == 'no'):
    customer_cities_df = city_data_df[(city_data_df['Rain Inches (last 3 hrs)'] == 0) | (city_data_df['Snow Inches (last 3 hrs)'] == 0)]

In [75]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = customer_cities_df.loc[(customer_cities_df["Max Temp"] <= max_temp) & (customer_cities_df["Max Temp"] >= min_temp)]
preferred_cities_df.dropna()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
2,2,Atuona,PF,2020-02-23 21:43:44,-9.80,-139.03,81.61,76,8,13.73,clear sky,0.0,0.0
4,4,Jacareacanga,BR,2020-02-23 21:43:45,-6.22,-57.75,80.55,79,100,3.24,overcast clouds,0.0,0.0
13,13,Kahului,US,2020-02-23 21:38:53,20.89,-156.47,80.60,54,20,6.93,few clouds,0.0,0.0
15,15,Sola,VU,2020-02-23 21:43:45,-13.88,167.55,83.14,77,20,14.70,few clouds,0.0,0.0
28,28,Hithadhoo,MV,2020-02-23 21:43:46,-0.60,73.08,83.41,76,92,13.69,overcast clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,531,San Juan,AR,2020-02-23 21:39:52,-31.54,-68.54,89.71,20,0,9.28,clear sky,0.0,0.0
546,546,Kattivakkam,IN,2020-02-23 21:44:15,13.22,80.32,80.01,83,20,9.78,mist,0.0,0.0
552,552,Moindou,NC,2020-02-23 21:44:16,-21.69,165.68,82.40,61,85,6.93,overcast clouds,0.0,0.0
561,561,Puerto Escondido,MX,2020-02-23 21:44:16,15.85,-97.07,87.80,79,20,5.82,few clouds,0.0,0.0


In [117]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [113]:
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [85]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [86]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [87]:
# Info Box template for map
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Description</dt><dd>It is {Description} with a high of {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [109]:
# Add a popup marker of cities for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.0)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))